In [ ]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


## 1.Xuất thư viện

In [ ]:
import pickle
import nltk
import re
import numpy as np
import pandas as pd
from __future__ import unicode_literals
import time
import tweepy

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.linear_model import LogisticRegression
from sklearn.multioutput import ClassifierChain
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import f1_score,precision_score,recall_score

from nltk import word_tokenize
from nltk.corpus import stopwords

## 2.Trích xuất data từ Twitter

In [ ]:
consumer_key = 'XXXXXXXXXXXXXXXXXXXXXX'
consumer_secret = 'XXXXXXXXXXXXXXXXXXXXXX'
access_key = 'XXXXXXXXXXXXXXXXXXXXXX'
access_secret = 'XXXXXXXXXXXXXXXXXXXXXX'

auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_key, access_secret)
api = tweepy.API(auth)

In [ ]:
def limitHandled(cursor):
  while True:
    try:
        yield cursor.next()
    except StopIteration:
      return  
    except tweepy.TweepError:
        time.sleep(15 * 60)

topics = ['Sony', 'politics', 'environment', 'technology', 'travel', 'sustainability', 
'economy', 'social media', 'sport', 'music', 'photography', 'healthcare', 'education', 'Sony and politics', 'Sony and environment', 'Sony and technology',
'Sony and travel', 'Sony and sustainability', 'Sony and economy', 'Sony and social media', 'Sony and sport', 'Sony and music', 'Sony and photography', 'Sony and education']
if __name__ == '__main__':
  for topic in topics:
    users = {}
    for tweet in limitHandled(tweepy.Cursor(api.search, q=(topic), count=100, lang='en', since = '2022-7-1').items(5000)):
      print(tweet.user.name)
      print(tweet.text)
      if tweet.user.id in users:
        users[tweet.user.id].append(tweet.text)  
      else:
        users[tweet.user.id] = []
        users[tweet.user.id].append(tweet.text)
    fileName = topic + '_train_data.data'
    print (fileName)
    userTweets = open(fileName,'ab+')
    pickle.dump(users,userTweets)
    userTweets.close()

Streaming output truncated to the last 5000 lines.
@theREkid99 @jrb @_XboxNews Sony bought one of the biggest published in the EU/UK when they first entered the conso… https://t.co/5u0NaNscCr
Marcus Osborne
@Dominicliddell Thanks for the info but I don’t and it’s not the internet because other devices are fine. It’s the… https://t.co/poT6y4bPdo
Hiroshi Yamaguchi
RT @ReutersScience: Japanese electronics and entertainment conglomerate Sony Group Corp said on Tuesday it has the technology to make human…
☪常岡浩介☪元容疑者
RT @ReutersScience: Japanese electronics and entertainment conglomerate Sony Group Corp said on Tuesday it has the technology to make human…
Motion Gaming
@_djcdjc @Bloodhunt Ps4 uses out dated storage technology and they currently have a sony exclusivity contract.
Stefan
@NintendoAmerica Thank you for Nintendo Switch OLED. A really amazing great wonderful Videogame Console. The best P… https://t.co/Gii6d1wre3
Quantlabs
According to a research 68% of Indian buyers are keen to tr

In [ ]:
fileName = ['Sony_train_data.data', 'politics_train_data.data', 'environment_train_data.data', 'technology_train_data.data', 'travel_train_data.data', 'sustainability_train_data.data', 
'economy_train_data.data', 'social media_train_data.data', 'sport_train_data.data', 'music_train_data.data', 'photography_train_data.data', 'healthcare_train_data.data', 'education_train_data.data', 'Sony and politics_train_data.data', 'Sony and environment_train_data.data', 'Sony and technology_train_data.data',
'Sony and travel_train_data.data', 'Sony and sustainability_train_data.data', 'Sony and economy_train_data.data', 'Sony and social media_train_data.data', 'Sony and sport_train_data.data', 'Sony and music_train_data.data', 'Sony and photography_train_data.data', 'Sony and education_train_data.data']

labelsFile = [['Sony'], ['politics'], ['ennvironment'], ['technology'], ['travel'], ['sustainability'],
              ['economy'],['social media'], ['sport'], ['music'], ['photography'], ['healthcare'], ['education'], ['Sony','politics'],
              ['Sony','environment'], ['Sony','technology'], ['Sony','travel'], ['Sony','sustainability'], ['Sony','economy'], ['Sony','social media'],
              ['Sony','sport'], ['Sony','music'], ['Sony','photography'], ['Sony','healthcare'], ['Sony','education']]
	      
def getCombineTweetsPerUser(tweetList):
  result = ''
  for tweet in tweetList:
    result = result + ' '
    result = result + tweet
  return result

def readPickleFiles():
  labelData = []
  contentData = []
  for index in range(len(fileName)):
    pkl_file = open('/content/gdrive/MyDrive/Data_Twitter/' + str(fileName[index]),'rb')
    getDict = pickle.load(pkl_file)
    for user in getDict:
      aggregatedDoc = getCombineTweetsPerUser(getDict[user])
      contentData.append(aggregatedDoc)
      labelData.append(labelsFile[index])
  return labelData,contentData
  
label, Content = readPickleFiles()
trainContent, testContent, labelTrain, labelTest = train_test_split(Content, label, test_size=.2, random_state=1)

## 3.Tiền xử lý

In [ ]:
nltk.download('stopwords')
nltk.download('punkt')
stopWords = stopwords.words('english')

def getOnlyWords(wordsList):
  pattern = re.compile('[a-zA-Z]+')
  result = []
  for word in wordsList:
    if pattern.match(word) != None:
      result.append(word)
  return result
  
def getWordsWithoutStop(sentence):
  sentence = sentence.lower()
  words = word_tokenize(sentence)
  withoutStopWords = [word for word in words if not word in stopWords]
  return getOnlyWords(withoutStopWords)
  
vectorizer = TfidfVectorizer(tokenizer = getWordsWithoutStop)
vectorizedTrainData = vectorizer.fit_transform(trainContent)
vectorizedTestData = vectorizer.transform(testContent)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
multiLabelBinary = MultiLabelBinarizer()
trainBinaryLabel = multiLabelBinary.fit_transform(labelTrain)
testBinaryLabel = multiLabelBinary.transform(labelTest)

## 4.Chạy mô hình Naive Bayes

In [ ]:
chains = [ClassifierChain(MultinomialNB(), order='random', random_state=i)
          for i in range(20)]
for chain in chains:
  chain.fit(vectorizedTrainData,trainBinaryLabel)

pred_train_chains = np.array([chain.predict(vectorizedTrainData) 
                             for chain in chains])


pred_train_ensemble = pred_train_chains.mean(axis=0)
threshold = [0.1, 0.2, 0.3, 0.4, 0.5]
optimal_threshold = 0.0
max_precision = 0.0

for thres in threshold:
  curPredictionsTrain = (pred_train_ensemble >= thres)
  precisionTrain = precision_score(trainBinaryLabel, curPredictionsTrain, average = 'micro')
  precisionTrain = precision_score(trainBinaryLabel, curPredictionsTrain, average = 'macro')
  if max_precision < precisionTrain:
    max_precision = precisionTrain
    optimal_threshold = thres


pred_test_chains = np.array([chain.predict(vectorizedTestData) 
                             for chain in chains])
pred_test_ensemble = pred_test_chains.mean(axis=0)
pred_test_ensemble = (pred_test_ensemble >= optimal_threshold)

/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Pr

## 5.Chạy mô hình Logistic Regression

In [ ]:
lr=LogisticRegression(penalty='l2',max_iter=500,C=1,random_state=0)


In [ ]:
chains = [ClassifierChain(lr, order='random', random_state=i)
          for i in range(20)]
for chain in chains:
  chain.fit(vectorizedTrainData,trainBinaryLabel)

pred_train_chains = np.array([chain.predict(vectorizedTrainData) 
                             for chain in chains])


pred_train_ensemble = pred_train_chains.mean(axis=0)
threshold = [0.1, 0.2, 0.3, 0.4, 0.5]
optimal_threshold = 0.0
max_precision = 0.0

for thres in threshold:
  curPredictionsTrain = (pred_train_ensemble >= thres)
  precisionTrain = precision_score(trainBinaryLabel, curPredictionsTrain, average = 'micro')
  precisionTrain = precision_score(trainBinaryLabel, curPredictionsTrain, average = 'macro')
  if max_precision < precisionTrain:
    max_precision = precisionTrain
    optimal_threshold = thres


pred_test_chains = np.array([chain.predict(vectorizedTestData) 
                             for chain in chains])
pred_test_ensemble2 = pred_test_chains.mean(axis=0)
pred_test_ensemble2 = (pred_test_ensemble2 >= optimal_threshold)

/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Pr

## 6.Thẩm định

### Tính điểm 2 mô hình theo recall, precision, f1

In [ ]:
# Điểm của mô hình Naive Bayes
precisionTest = precision_score(testBinaryLabel, pred_test_ensemble, average = 'micro')
f1Test = f1_score(testBinaryLabel, pred_test_ensemble, average = 'micro')
recallTest = recall_score(testBinaryLabel, pred_test_ensemble, average = 'micro')


precisionTest2 = precision_score(testBinaryLabel, pred_test_ensemble, average = 'macro')
f1Test2 = f1_score(testBinaryLabel, pred_test_ensemble, average = 'macro')
recallTest2 = recall_score(testBinaryLabel, pred_test_ensemble, average = 'macro')


In [ ]:
# Điểm của mô hình Logistic Regression
precisionTest_lr = precision_score(testBinaryLabel, pred_test_ensemble2, average = 'micro')
f1Test_lr = f1_score(testBinaryLabel, pred_test_ensemble2, average = 'micro')
recallTest_lr = recall_score(testBinaryLabel, pred_test_ensemble2, average = 'micro')


precisionTest2_lr = precision_score(testBinaryLabel, pred_test_ensemble2, average = 'macro')
f1Test2_lr = f1_score(testBinaryLabel, pred_test_ensemble2, average = 'macro')
recallTest2_lr = recall_score(testBinaryLabel, pred_test_ensemble2, average = 'macro')


### Theo average = 'micro'

In [ ]:
from tabulate import tabulate

table_heading = [ "Score", "Naive Bayes","Logistic Regression"]

table_data = [["recall", recallTest,recallTest_lr], ["Precision", precisionTest,precisionTest_lr], ["f1_score",f1Test,f1Test_lr]]

print("Điểm theo average = 'micro'")
print(tabulate(table_data, headers=table_heading, tablefmt="fancy_grid"))

Điểm theo average = 'micro'
╒═══════════╤═══════════════╤═══════════════════════╕
│ Score     │   Naive Bayes │   Logistic Regression │
╞═══════════╪═══════════════╪═══════════════════════╡
│ recall    │      0.416681 │              0.755834 │
├───────────┼───────────────┼───────────────────────┤
│ Precision │      0.993611 │              0.969835 │
├───────────┼───────────────┼───────────────────────┤
│ f1_score  │      0.587139 │              0.849565 │
╘═══════════╧═══════════════╧═══════════════════════╛


### Theo average = 'macro'

In [ ]:

table_heading = [ "Score", "Naive Bayes","Logistic Regression"]

table_data = [["recall", recallTest2,recallTest2_lr], ["Precision", precisionTest2,precisionTest2_lr], ["f1_score",f1Test2,f1Test2_lr]]

print("Điểm theo average = 'macro'")
print(tabulate(table_data, headers=table_heading, tablefmt="fancy_grid"))

Điểm theo average = 'macro'
╒═══════════╤═══════════════╤═══════════════════════╕
│ Score     │   Naive Bayes │   Logistic Regression │
╞═══════════╪═══════════════╪═══════════════════════╡
│ recall    │      0.380997 │              0.699077 │
├───────────┼───────────────┼───────────────────────┤
│ Precision │      0.921167 │              0.899579 │
├───────────┼───────────────┼───────────────────────┤
│ f1_score  │      0.530539 │              0.78591  │
╘═══════════╧═══════════════╧═══════════════════════╛


## 7.Dữ liệu dự đoán đa nhãn

In [ ]:
#Mô hình Naive Bayes
pred_test_ensemble_df = pd.DataFrame(pred_test_ensemble)
pred_test_ensemble_df.head(10)


,0,1,2,3,4,5,6,7,8,9,10,11,12,13
0,False,False,False,False,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,False,False,False,True
5,False,False,False,False,False,False,False,False,False,False,False,False,False,False
6,True,False,False,False,False,False,False,False,False,False,False,False,False,False
7,False,False,False,False,False,False,False,False,False,False,False,False,False,True
8,False,False,False,False,False,False,False,False,False,False,False,False,False,False
9,False,False,False,False,False,False,False,False,False,False,True,False,False,False


In [ ]:
pred_test_ensemble_df.to_excel("predict.xlsx")

In [ ]:
#Mô hình Logistic Regression
pred_test_ensemble_df = pd.DataFrame(pred_test_ensemble2)
pred_test_ensemble_df.head(10)


,0,1,2,3,4,5,6,7,8,9,10,11,12,13
0,False,False,False,False,False,False,False,False,False,False,False,False,False,True
1,False,False,False,False,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,True,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,False,False,False,True
5,False,False,False,False,False,False,False,False,False,False,False,False,True,False
6,True,False,False,False,False,False,False,False,False,False,False,False,False,False
7,False,False,False,False,False,False,False,False,False,False,False,False,False,True
8,True,False,False,False,False,False,True,False,False,False,False,False,False,False
9,False,False,False,False,False,False,False,False,False,False,True,False,False,False


In [ ]:
pred_test_ensemble_df.to_excel("predict2.xlsx")